# Applied Data Science Capstone Assignment Week 3

## Table of Content

1. <a href="#part1">Part 1 - Scraping and transforming table<br></a>
    1. <a href="#part1a">Import necessary libraries<br></a>
    1. <a href="#part1b">Scraping table from wikipedia and reading it into a dataframe<br></a>
    1. <a href="#part1c">Transforming table<br></a>
2. <a href="#part2">Part 2 - Adding geo-locational data<br></a>
    2. <a href="#part2a">Import necessary libraries<br></a>
    2. <a href="#part2b">Fetching geo-locational data<br></a>
    2. <a href="#part2c">Creating a new dataframe with the combined information<br></a>
3. <a href="#part3">Part 3 - Exploring and clustering neighborhoods<br></a>
    3. <a href="#part3a">Import necessary libraries<br></a>
    3. <a href="#part3b">Visualize toronto and its neighbourhoods<br></a>
    3. <a href="#part3c">Explore neighbourhoods of toronto<br></a>
    3. <a href="#part3d">Analyze the neighborhoods<br></a>
    3. <a href="#part3e">Cluster neighborhoods<br></a>

1. [Part 1 - Scraping and transforming table](#part1)
    1. [Import necessary libraries](#part1a)
    1. [Scraping table from wikipedia and reading it into a dataframe](#part1b)
    1. [Transforming table](#part1c)
2. [Part 2 - Adding geo-locational data](#part2)
    2. [Import necessary libraries](#part2a)

## <a class="anchor" id="part1">1) Part 1 - Scraping and transforming table</a>

<h3 id="part1a">1A) Import necessary libraries</h3>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h3 id="part1b">1B) Scraping table from wikipedia and reading it into a dataframe</h3>

0th index is taken beacuse this is the first of three tables on this page

In [2]:
df_tor = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

<h3 id="part1c">1C) Transforming table</h3>

Change the column names accordingly

In [3]:
df_tor.columns = ["PostalCode", "Borough", "Neighborhood"]

Drop rows that have no borough assigned

In [4]:
df_tor = df_tor[df_tor["Borough"] != "Not assigned"]

If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [5]:
df_tor[df_tor["Neighborhood"] == "Not assigned"]["Neighborhood"] = df_tor[df_tor["Neighborhood"] == "Not assigned"]["Borough"]

Reset index to start with 0

In [6]:
df_tor.reset_index(drop=True, inplace=True)

Neighborhoods have already been combined on the wikipedia page so there is no need to do that here

In [51]:
print("Number of unique postal codes:", df_tor["PostalCode"].nunique())
print("Number of rows:", df_tor.shape[0])

Number of unique postal codes: 103
Number of rows: 103


In [7]:
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Print the number of rows of the dataframe

In [8]:
print(f"The dataframe has {df_tor.shape[0]} rows and {df_tor.shape[1]} columns")

The dataframe has 103 rows and 3 columns


<h2 id="part2">2) Part 2 - Adding geo-locational data</h2>

<h3 id="part2a">2A) Import necessary libraries</h3>

In [9]:
#%conda install -c conda-forge geocoder

In [10]:
import geocoder  # api to get location data
import urllib.request  # library for downloading files from web

<h3 id="part2b">2B) Fetching geo-locational data</h3>

Trying to get the data from the geocoder api

In [11]:
for index in df_tor.index:
    # get postal code for that row
    postal_code = df_tor.iloc[index]["PostalCode"]
    
    print("Current index:", index, "Postal code:", postal_code)
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while lat_lng_coords is None:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    # add long- and latitude to dataframe
    df_tor.iloc[index]["Latitude"] = lat_lng_coords[0]
    df_tor.iloc[index]["Longitude"] = lat_lng_coords[1]
print("Geo data sucessfully added!")

Current index: 0 Postal code: M3A


KeyboardInterrupt: 

Since retrieving the data via the api did not work (never got past first row) the csv file will be used instead

In [12]:
url = 'http://cocl.us/Geospatial_data'
filename = 'geospatial_data.csv'
urllib.request.urlretrieve(url, filename)  # download file from url and save it locally

('geospatial_data.csv', <http.client.HTTPMessage at 0x2ca1a819c48>)

Read csv file into a dataframe

In [13]:
df_tor_geo = pd.read_csv(filename)
# Change column names to match other dataframe
df_tor_geo.columns = ["PostalCode", "Latitude", "Longitude"]

In [14]:
df_tor_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3 id="part2c">2C) Creating a new dataframe with the combined information</h3>

Combine the two tables in an SQL left join like manner

In [15]:
df_tor_comb = pd.merge(df_tor, df_tor_geo, how="left", on="PostalCode")

Inspect the new table

In [16]:
df_tor_comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [17]:
df_tor_comb.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h2 id="part3">3) Part 3 - Exploring and clustering neighborhoods</h2>

<h3 id="part3a">3A) Import necessary libraries</h3>

In [18]:
#%conda install -c conda-forge folium=0.5.0 --yes

In [19]:
#%conda install -c conda-forge geopy --yes

In [20]:
import numpy as np

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

<h3 id="part3b">3B) Visualize toronto and its neighbourhoods</h3>

Use geopy to get the long- and latitude of toronto

In [21]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top

In [38]:
# create map of Toronto using latitude and longitude values
lat_offset = .065 # little offset to fit it better in the window
map_toronto = folium.Map(location=[latitude+lat_offset, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor_comb['Latitude'], df_tor_comb['Longitude'], df_tor_comb['Borough'], df_tor_comb['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3 id="part3c">3C) Explore neighbourhoods of toronto</h3>

Define Foursquare credentials and version

In [23]:
CLIENT_ID = 'MK0SEV54RIMKQW3Y514Z3QLU1R24CAEYS3KUVRYX3C4QNANJ' # your Foursquare ID
CLIENT_SECRET = 'QWAKFPQFROEBEBQ1BSMJ4X0ZSAYVBDJJLCODBFLSHMXLDCPA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MK0SEV54RIMKQW3Y514Z3QLU1R24CAEYS3KUVRYX3C4QNANJ
CLIENT_SECRET:QWAKFPQFROEBEBQ1BSMJ4X0ZSAYVBDJJLCODBFLSHMXLDCPA


Define a helper function to explore all the neighborhoods

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?"\
        f"&client_id={CLIENT_ID}"\
        f"&client_secret={CLIENT_SECRET}"\
        f"&v={VERSION}"\
        f"&ll={lat},{lng}"\
        f"&radius={radius}"\
        f"&limit={LIMIT}"
        
     
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Store the venue information in a new dataframe

In [47]:
# define needed variables
LIMIT = 100
radius = 500

toronto_venues = getNearbyVenues(
    names=df_tor_comb["Neighborhood"],
    latitudes=df_tor_comb["Latitude"],
    longitudes=df_tor_comb["Longitude"]
)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

KeyError: 'groups'

Inspect the new dataframe

In [52]:
print("Shape of toronto_venus dataframe:", toronto_venues.shape)
toronto_venues.head(10)

Shape of toronto_venus dataframe: (767, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downsview,43.761631,-79.520999,Durante's No Frills,43.758178,-79.519680,Grocery Store
1,Downsview,43.761631,-79.520999,LCBO,43.759257,-79.519454,Liquor Store
2,Downsview,43.761631,-79.520999,Driftwood community centre,43.765680,-79.519706,Athletics & Sports
3,Downsview,43.761631,-79.520999,Planet Fitness,43.757538,-79.519610,Gym / Fitness Center
4,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
5,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
6,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
7,Roselawn,43.711695,-79.416936,Dr.Paul Hodges MIP,43.710634,-79.415810,Health & Beauty Service
8,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service
9,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


Check how many venues were returned for each neighborhood

In [53]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Canada Post Gateway Processing Centre,13,13,13,13,13,13
Church and Wellesley,75,75,75,75,75,75
"Clarks Corners, Tam O'Shanter, Sullivan",11,11,11,11,11,11
Davisville,32,32,32,32,32,32
Davisville North,8,8,8,8,8,8


Get unique categories

In [56]:
print(f"There are {len(toronto_venues['Venue Category'].unique())} unique categories.")

There are 203 unique categories.


<h3 id="part3d">3D) Analyze the neighborhoods</h3>

Create one-hot tensors for each category

In [58]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print("Shape of new dataframe:", toronto_venues.shape)
toronto_onehot.head()

Shape of new dataframe: (767, 7)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Downsview,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Downsview,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Downsview,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group rows by neighborhood and calculate the mean of the frequency of each category

In [60]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print("Shape of new dataframe:", toronto_grouped.shape)
toronto_grouped

Shape of new dataframe: (42, 204)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Agincourt,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00000

Print the top 5 most common venues of each neighborhood

In [62]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4          Afghan Restaurant  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.29
1           Pub  0.14
2           Gym  0.14
3   Coffee Shop  0.14
4  Skating Rink  0.14


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1       Garden Center  0.06
2              Garden  0.06
3             Brewery  0.06
4          Skate Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3  Airport Food Court  0.06
4    Sculpture Garden  0.06


----Canada Post Gateway Processing Centre----
           

First, write a function to sort the venues in descending order then store it in a dataframe

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe to display the top 10 venues for each neighborhood

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Yoga Studio,Doner Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Gym,Coffee Shop,Pub,Skating Rink,Yoga Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden,Park,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar
4,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Burrito Place,Gym,Intersection,Middle Eastern Restaurant,American Restaurant,Mediterranean Restaurant,Fried Chicken Joint,Sandwich Place


<h3 id="part3e">3E) Cluster neighborhoods</h3>